$$
\newcommand{\F}{\mathbb{F}}
\newcommand{\Fq}{\F_q}
\newcommand{\K}{\mathbb{K}}
\newcommand{\V}{\mathbb{V}}
$$

We're working with $\Fq$ for $q$ a power of 2. $\K$ is an extension of degree $n$, represented as polynomials from $\Fq[X]/(P)$, where $P$ is an irreducible polynomial of degree $n$ over $\Fq$.

We introduce some sage weirdness so we can do symbolic calculations later on. More in particular, we introduce $n$ extra "monomials", such that they are actually properly reduced as well

In [1]:
q = 2^2
n = 3
F = PolynomialRing(GF(q, name='ɑ'), ['X'] + [f'x{i}' for i in range(n)])
X, *dummies = F.gens()
alpha = F.base().gen()
poly = F.base()['X'].irreducible_element(n)
poly = F.base()['X'].gen()^3 + alpha
assert poly.is_irreducible()
K = F.quotient_ring([poly.subs({poly.variables()[0]: X})] + [d^q + d for d in dummies], names=[str(t) for t in ['X', *dummies]])
X, *_ = K.gens()

To have a correspondence between the vector space $\V = \Fq^n$ and $\K$, we define the basis $(\beta_i = X^i)_{0 \le i < n}$

In [2]:
beta = [X^i for i in range(n)]
beta

[1, X, X^2]

This, in turn also easily allows us to do the following map from vector-land to field-land in sage

We write $$\phi : \K \to \V : a_0 + a_1X + a_2 X^2 + a_3 X^3 + a_4 X^4 \mapsto (a_0, a_1, a_2, a_3, a_4)$$ and $$\phi^{-1} : \V \to \K : (a_i)_{0 \le i < 5} \mapsto \sum_{i = 0}^4 a_i \beta_i$$ 
Note that this definition for $\phi$ works for this particular $\beta$ but would not work in general, though the notation could remain the same for full generality

In [3]:
def phi_inv(x):
    return sum(b*y for b, y in zip(beta, x))

def phi(x):
    # https://ask.sagemath.org/question/52594/how-to-get-the-coefficient-of-a-multivariate-polynomial-with-respect-to-a-specific-variable-and-degree-in-a-quotient-ring/
    xl = x.lift()
    # Note, this *really* only works for this particular \beta
    return vector(F, [xl.coefficient({X: i}) for i in range(n)])

K_el = phi_inv(vector(F, [alpha^2, 0, alpha]))
V_el = phi(K_el)
K_el, V_el

((ɑ)*X^2 + (ɑ + 1), ((ɑ + 1), 0, (ɑ)))

Then some further toy parameters:

In [4]:
theta = 2
h = q^theta + 1
h_inv = inverse_mod(h, q^n - 1)

And the affine transformations

In [5]:
A = Matrix(F, [1, 0, alpha, 0, 0, alpha + 1, alpha, alpha, alpha], nrows=n)
c = vector(F, [1, alpha, 1])
L1 = lambda x: vector(F, A*x + c)
L1_inv = lambda x: vector(F, A^-1 * (x - c))

B = Matrix(F, [1, alpha, alpha + 1, 0, 1, alpha + 1, alpha, alpha, alpha + 1 ], nrows=n)
d = vector(F, [alpha + 1, 0, alpha + 1])
L2 = lambda x: vector(F, B * x + d)
L2_inv = lambda x: vector(F, B^-1 * (x - d))

To obtain the public relations $\bar{x} \to \bar{y}$, we first calculate $\bar{u}$ in terms of $\bar{x}$, bring it over to $\mathbf{u} \in \K$ through $\phi^{-1}$, calculate $\mathbf{v} = \mathbf{u}^h$ and bring that back to $\bar{v} = \phi(\mathbf{v})$

In [6]:
x_bar = vector(F, dummies)
psi = lambda x: x^h
y_bar = L2(phi(psi(phi_inv(L1(x_bar)))))
print("\n".join(map(str, L2(phi(psi(phi_inv(L1(x_bar))))))))

x0^2 + (ɑ + 1)*x0*x1 + (ɑ + 1)*x1^2 + (ɑ)*x0*x2 + (ɑ + 1)*x1*x2 + (ɑ + 1)*x2^2 + x0 + (ɑ)*x1 + (ɑ)
x0^2 + (ɑ + 1)*x0*x1 + (ɑ)*x1^2 + (ɑ + 1)*x0*x2 + x1*x2 + (ɑ)*x2^2 + (ɑ)*x0 + (ɑ + 1)*x1 + (ɑ + 1)
(ɑ)*x0^2 + (ɑ + 1)*x0*x1 + (ɑ + 1)*x1^2 + (ɑ + 1)*x0*x2 + (ɑ)*x1*x2 + (ɑ)*x0 + x1 + (ɑ + 1)


The public key then simply consists of these multivariate polynomial relations

The private key consists of our random masks $L_1$ and $L_2$ (or their inverses, practically speaking), and the inverse mapping for the $h$th power, which is simply $h^{-1} \pmod{q^n - 1}$

In [7]:
pubkey = y_bar
privkey = (L1_inv, L2_inv, h_inv)

To encrypt a message, we simply inject our plaintext into the public polynomials

In [8]:
def encrypt(pubkey, message):
    assert len(pubkey) == len(message)
    d = {x: m for x, m in zip(dummies, message)}
    return vector(F, [f.substitute(d) for f in pubkey])
ciphertext = encrypt(pubkey, vector(F, [alpha, alpha, alpha]))
ciphertext

(1, 1, 0)

To decrypt a plaintext again, we apply the individual inverse steps as part of the private key

In [9]:
def decrypt(privkey, ct):
    L1_inv, L2_inv, h_inv = privkey
    return L1_inv(phi(phi_inv(L2_inv(ct))^h_inv))
decrypt(privkey, ciphertext)

((ɑ), (ɑ), (ɑ))

-------------------------------
Now, let's see if we can get the attack to work